[View in Colaboratory](https://colab.research.google.com/github/malinenimaurya/Deep-Learning/blob/master/transfer_learning_cifar_keras.ipynb)

In [3]:
import numpy as np
from matplotlib import pyplot as plt

import keras
from keras.datasets import cifar10
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import to_categorical


# In[4]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_examples = 50000


# In[7]:

X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])


# In[13]:

X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

Y1_train = to_categorical(np.asarray(Y1_train))
Y1_test = to_categorical(np.asarray(Y1_test))

Y2_train = to_categorical(np.asarray(Y2_train))
Y2_test = to_categorical(np.asarray(Y2_test))


# In[18]:

print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)


# In[26]:

split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]


# In[30]:

model = Sequential()

model.add(Convolution2D(32, 5, 5, input_shape=(32, 32, 3), activation='relu'))
model.add(Convolution2D(16, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(8, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.42))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# In[ ]:

import time, datetime

start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
         nb_epoch=10,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)




(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 5) (5000, 5)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(32, 32, 3..., activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 77s - loss: 1.1830 - acc: 0.4986 - val_loss: 1.0479 - val_acc: 0.5692
Epoch 2/10
 - 77s - loss: 0.9972 - acc: 0.5911 - val_loss: 0.9394 - val_acc: 0.6200
Epoch 3/10
 - 77s - loss: 0.9307 - acc: 0.6243 - val_loss: 0.8809 - val_acc: 0.6606
Epoch 4/10
 - 76s - loss: 0.8975 - acc: 0.6411 - val_loss: 0.8556 - val_acc: 0.6600
Epoch 5/10
 - 77s - loss: 0.8695 - acc: 0.6518 - val_loss: 0.8711 - val_acc: 0.6580
Epoch 6/10
 - 77s - loss: 0.8371 - acc: 0.6652 - val_loss: 0.8518 - val_acc: 0.6618
Epoch 7/10
 - 77s - loss: 0.8286 - acc: 0.6698 - val_loss: 0.8286 - val_acc: 0.6704
Epoch 8/10
 - 76s - loss: 0.7989 - acc: 0.6887 - val_loss: 0.8113 - val_acc: 0.6862
Epoch 9/10
 - 77s - loss: 0.7764 - acc: 0.6954 - val_loss: 0.7800 - val_acc: 0.6972
Epoch 10/10
 - 77s - loss: 0.7506 - acc: 0.7056 - val_loss: 0.7580 - val_acc: 0.7010


 -------------------- 

Time taken for first training:  0:12:45.928228

 -------------------- 




In [4]:
# In[31]:

for l in model.layers[:6]:
    l.trainable = False   

# In[ ]:

trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# In[ ]:

start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, nb_epoch=10, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               102528    
__________

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 30s - loss: 0.9334 - acc: 0.6449 - val_loss: 0.7724 - val_acc: 0.7056
Epoch 2/10
 - 31s - loss: 0.8090 - acc: 0.6901 - val_loss: 0.7480 - val_acc: 0.7186
Epoch 3/10
 - 31s - loss: 0.7782 - acc: 0.7046 - val_loss: 0.7398 - val_acc: 0.7166
Epoch 4/10
 - 31s - loss: 0.7579 - acc: 0.7127 - val_loss: 0.7195 - val_acc: 0.7254
Epoch 5/10
 - 31s - loss: 0.7320 - acc: 0.7205 - val_loss: 0.6799 - val_acc: 0.7424
Epoch 6/10
 - 31s - loss: 0.7123 - acc: 0.7260 - val_loss: 0.6743 - val_acc: 0.7424
Epoch 7/10
 - 31s - loss: 0.6976 - acc: 0.7387 - val_loss: 0.6595 - val_acc: 0.7566
Epoch 8/10
 - 31s - loss: 0.6782 - acc: 0.7416 - val_loss: 0.6513 - val_acc: 0.7536
Epoch 9/10
 - 31s - loss: 0.6667 - acc: 0.7496 - val_loss: 0.6456 - val_acc: 0.7560
Epoch 10/10
 - 31s - loss: 0.6541 - acc: 0.7535 - val_loss: 0.6485 - val_acc: 0.7576


 -------------------- 

Time taken for final training:  0:05:08.480310

 -------------------- 


